# Introduction to De-noising Diffusion Probabilistic Models

In [ ]:
# import numpy as onp
import jax.random as random

rng = random.PRNGKey(2022)

In [ ]:
def sample_sphere(n_points: int) -> Array:
    alphas = jnp.linspace(0, 2 * jnp.pi * (1 - 1 / n_points), n_points)
    xs = jnp.cos(alphas)
    ys = jnp.sin(alphas)
    mf = jnp.stack([xs, ys], axis=1)
    return mf


sphere = sample_sphere(100) * 0.5 + 0.7
plt.scatter(sphere[:, 0], sphere[:, 1])